# Prepare Open AI client object

In [23]:
from openai import OpenAI

In [24]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [25]:
client=OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Prepare documents for information retrieval

In [8]:
import minsearch

In [9]:
import json

In [10]:
with open('documents.json', 'rt') as f_in:
    docs_raw=json.load(f_in)

In [12]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [17]:
q ='the course has already started, can I still enroll?'

In [18]:
index.fit(documents)

In [21]:
boost={'question': 3.0, 'section': 0.5 } # By default, all parameters have value of 1.
#Increasing or decreasing this value will change its relevance during the search process.

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course': 'data-engineering-zoomcamp'}
)

In [22]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Generate answers from OpenAI GPT

Test generation without context

In [26]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':q}]
)

In [28]:
response.choices[0].message.content

"It depends on the course and the institution offering it. Many institutions have a late enrollment policy that allows students to join within a certain period after the course has started. Here are a few steps you can take:\n\n1. **Check the Course Policy**: Look at the syllabus or any official communication you have about the course to see if there is information on late enrollment.\n   \n2. **Contact the Instructor**: Reach out to the course instructor directly to ask if you can still enroll. Express your interest and ask if there are any special considerations or requirements you need to meet.\n\n3. **Contact the Registrar or Admissions Office**: These offices can provide information on late enrollment policies and help guide you through the necessary steps if late enrollment is permitted.\n\n4. **Catch Up on Missed Material**: If you are allowed to enroll late, make sure to ask about any material you missed and any assignments you need to catch up on.\n\n5. **Consider Self-paced o

Test generation with context

In [37]:
prompt_template="""
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts of the CONTEXT in the answer. 
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}

""".strip()

In [31]:
context=''

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [34]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [40]:
prompt = prompt_template.format(question=q, context=context).strip()

In [41]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
)

In [43]:
print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
